<a href="https://colab.research.google.com/github/Success-Vera/Gaussian-Mixture-Models/blob/main/master.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reading Our Custom Libraries

## Run these cells without changing anything

In [3]:
user = "Success-Vera"
repo = "Gaussian-Mixture-Models"

prepfile='preprocessing.py' #our preprocessing library
svd='SVD.py' #our svd library

url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{pyfile}"
!wget --no-cache --backups=1 {url}

url2 = f"https://raw.githubusercontent.com/{user}/{repo}/main/{prepfile}"
!wget --no-cache --backups=1 {url2}

url3 = f"https://raw.githubusercontent.com/{user}/{repo}/main/{svd}"
!wget --no-cache --backups=1 {url3}

--2022-04-22 09:15:56--  https://raw.githubusercontent.com/Success-Vera/Gaussian-Mixture-Models/main/frequency.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3380 (3.3K) [text/plain]
Failed to rename frequency.py to frequency.py.1: (2) No such file or directory
Saving to: ‘frequency.py’

frequency.py        100%[===================>]   3.30K  --.-KB/s    in 0s      

2022-04-22 09:15:56 (47.1 MB/s) - ‘frequency.py’ saved [3380/3380]

--2022-04-22 09:15:56--  https://raw.githubusercontent.com/Success-Vera/Gaussian-Mixture-Models/main/preprocessing.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:4

In [4]:
import subprocess

result = subprocess.run(["wget", "--no-cache", "--backups=1", url], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
result2 = subprocess.run(["wget", "--no-cache", "--backups=1", url2], stderr=subprocess.PIPE, stdout=subprocess.PIPE)
result3 = subprocess.run(["wget", "--no-cache", "--backups=1", url2], stderr=subprocess.PIPE, stdout=subprocess.PIPE)

# print(result2.stderr.decode("utf-8"))

In [9]:
!unzip '/content/drive/MyDrive/GMM/IMDB Dataset.csv.zip'

Archive:  /content/drive/MyDrive/GMM/IMDB Dataset.csv.zip
  inflating: IMDB Dataset.csv        


## Importing Libraries

In [32]:
from preprocessing import forward
import SVD 
import pandas as pd
import numpy as np


## Data Preprocessing



In [10]:
data=pd.read_csv('/content/IMDB Dataset.csv')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [11]:
data['clean_text']=data['review'].apply(lambda X: forward(X))
data.head()

,review,sentiment,clean_text
0,One of the other reviewers has mentioned that ...,positive,One review ment watc 1 Oz episod ll hook Thei ...
1,A wonderful little production. <br /><br />The...,positive,A wond littl product br br The fil techniqu un...
2,I thought this was a wonderful way to spend ti...,positive,I thought wond wai spend time hot summ weekend...
3,Basically there's a family where a little boy ...,negative,Bas famili littl boi Jake think zombi closet p...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,Pett Mattei Love Time Monei visu stun film wat...


In [14]:
data.drop('review',inplace=True,axis=1)

In [15]:
data['sentiment'].replace({'negative':0,"positive":1},inplace=True)

In [16]:
data_set=data[['clean_text','sentiment']].copy()
data_set.head()

,clean_text,sentiment
0,One review ment watc 1 Oz episod ll hook Thei ...,1
1,A wond littl product br br The fil techniqu un...,1
2,I thought wond wai spend time hot summ weekend...,1
3,Bas famili littl boi Jake think zombi closet p...,0
4,Pett Mattei Love Time Monei visu stun film wat...,1


In [17]:
index = data_set.index
data_set['random_number'] = np.random.randn(len(index))
train = data_set[data_set['random_number'] <= 0.8]
test = data_set[data_set['random_number'] > 0.8]


In [18]:
data_set.head()

,clean_text,sentiment,random_number
0,One review ment watc 1 Oz episod ll hook Thei ...,1,-0.209437
1,A wond littl product br br The fil techniqu un...,1,-0.518295
2,I thought wond wai spend time hot summ weekend...,1,1.311180
3,Bas famili littl boi Jake think zombi closet p...,0,0.238930
4,Pett Mattei Love Time Monei visu stun film wat...,1,-0.137900


In [22]:
train.drop('random_number',inplace=True,axis=1)
test.drop('random_number',inplace=True,axis=1)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
train.shape,test.shape

((39596, 2), (10404, 2))

In [25]:
test.head()

,clean_text,sentiment
2,I thought wond wai spend time hot summ weekend...,1
12,So im big fan Boll work mani I enjoi movi Post...,0
13,The cast plai Shakespear br br Shakespear lost...,0
15,Kind drawn erot scen real one amateurish unbel...,0
20,Aft success Die Hard sequel surpris real 1990 ...,1


## Word Embedding with TF-IDF

In [56]:
import re
class FrequencyEcoder:

    def __init__(self, data):
        """These data here has to be cleaned without those special symbols."""
        self.data = data

    """This function has task of computing the frequency of the word in each row of the file and returns the row as dictionary of word as key and 
    frequency as value.
    """

    def remove_char(text):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)

    def frequency_computer(self, string):

        string = string.split()

        dict_freq = {}

        freq_list = []

        for i in string:
            if i not in freq_list:
                #             print(i)
                freq_list.append(i)
        for j in range(0, len(freq_list)):
            dict_freq[freq_list[j]] = string.count(freq_list[j])
        return dict_freq

    """This function is for changing each row of the file into dictionary words as keys and their repetitions as values. It will be usefully to
    the time of replacing the word by its total frequency in all file.
    """

    def row_to_dict(self):
        inputs = self.data
        for index, item in inputs.items():
            inputs[index] = self.remove_char(item).replace('br br', ' ')
            for _ in inputs[index]:
                if type(_) == int:
                    inputs[index] = item.replace('_', ' ')

            inputs[index] = frequency_computer(inputs[index])
        return inputs


"""Convert each row of the file into the list of words 
  --------------------------------------------------------
  This function has objective of transforming the row of file into the list of words in each row
    """


def conv(self, stock_str):
    n = int(input('Enter the number of rows modify: '))

    for i in range(n):
        print(stock_str[i])
        liste = stock_str[i].split()

        #         st=','.join([item  for item in liste])
        stock_str[i] = liste

    return stock_str[:n]


"""Combination of two dictionaries to get one with updated values, here we take two rows that 
contains dictionaris and put them together to get one dictionary of updated keys and value
----------
This function will be useful to the time of finding frequency of each word in a whole document
"""


def dict_combine_2(self, dic1, dic2):
    return dict(sorted({k: dic1.get(k, 0) + dic2.get(k, 0) for k in set(dic1) | set(dic2)}.items()))


"""ENCODING DOCUMENT 
---------------------
  This function is going to use that function which combine two dictionaries and use it to get a frequency of each word in full document.
            """


def final_encoder(self):
    dict_store = self.row_to_dict()

    start = dict_store[0]

    for i in range(1, len(dict_store)):
        var = self.dict_combine_2(start, dict_store[i])
        start = var
    return start


"""Transform the  file
-----------------------------
This function is tasked to use a created functions, and produces the encoded file using the frequencies of words in a whole file.
"""


def transform(self):
    inputs = self.row_to_dict()
    temp = self.conv(self.data)

    ff = self.final_encoder(inputs[:5])

    final = []
    new_df = temp.copy()
    for idx, item in enumerate(temp):
        final = []
        for x in item:
            if x in ff.keys():
                final.append(ff[x])
        new_df[idx] = final

    return new_df

In [ ]:
vectorizer= FrequencyEcoder()
tf_x_train = vectorizer.transform(train['clean_text'])
tf_x_test = vectorizer.transform(test['clean_text'])

## Dimension Reduction Using SVD

In [33]:
new_data=SVD.svd_decompose(tf_x_train,6)
new_data_test=svd_decompose(tf_x_test,6)

In [36]:
new_data_test.shape,new_data.shape

((10404, 6), (1000, 6))

## Gaussian Mixture Model

In [69]:
class GMM:
    """
        This class is the implementation of the Gaussian Mixture Models
        inspired by sci-kit learn implementation.
    """

    def __init__(self, n_components, max_iter=100, comp_names=None):
        """
            This functions initializes the model by seting the following paramenters:
                :param n_components: int
                    The number of clusters in which the algorithm must split
                    the data set
                :param max_iter: int, default = 100
                    The number of iteration that the algorithm will go throw to find the clusters
                :param comp_names: list of strings, default=None
                    In case it is setted as a list of string it will use to
                    name the clusters
        """
        self.n_componets = n_components
        self.max_iter = max_iter
        if comp_names == None:
            self.comp_names = [f"comp{index}" for index in range(self.n_componets)]
        else:
            self.comp_names = comp_names
        # pi list contains the fraction of the dataset for every cluster
        self.pi = [1 / self.n_componets for comp in range(self.n_componets)]

    def multivariate_normal(self, X, mean_vector, covariance_matrix):
        """
            This function implements the multivariat normal derivation formula,
            the normal distribution for vectors it requires the following parameters
                :param X: 1-d numpy array
                    The row-vector for which we want to calculate the distribution
                :param mean_vector: 1-d numpy array
                    The row-vector that contains the means for each column
                :param covariance_matrix: 2-d numpy array (matrix)
                    The 2-d matrix that contain the covariances for the features
        """
        return (2 * np.pi) ** (-len(X) / 2) * np.linalg.det(covariance_matrix) ** (-1 / 2) * np.exp(
            -np.dot(np.dot((X - mean_vector).T, np.linalg.inv(covariance_matrix)), (X - mean_vector)) / 2)

    def fit(self, X):
        """
            The function for training the model
                :param X: 2-d numpy array
                    The data must be passed to the algorithm as 2-d array,
                    where columns are the features and the rows are the samples
        """
        # Spliting the data in n_componets sub-sets
        new_X = np.array_split(X, self.n_componets)
        # Initial computation of the mean-vector and covarience matrix
        self.mean_vector = [np.mean(x, axis=0) for x in new_X]
        self.covariance_matrixes = [np.cov(x.T) for x in new_X]
        # Deleting the new_X matrix because we will not need it anymore
        del new_X
        for iteration in range(self.max_iter):
            ''' --------------------------   E - STEP   -------------------------- '''
            # Initiating the r matrix, evrey row contains the probabilities
            # for every cluster for this row
            self.r = np.zeros((len(X), self.n_componets))
            # Calculating the r matrix
            for n in range(len(X)):
                for k in range(self.n_componets):
                    self.r[n][k] = self.pi[k] * self.multivariate_normal(X[n], self.mean_vector[k],
                                                                         self.covariance_matrixes[k])
                    self.r[n][k] /= sum(
                        [self.pi[j] * self.multivariate_normal(X[n], self.mean_vector[j], self.covariance_matrixes[j])
                         for j in range(self.n_componets)])
                    print(self.multivariate_normal(X[n], self.mean_vector[k], self.covariance_matrixes[k]))
            # Calculating the N
            N = np.sum(self.r, axis=0)
            ''' --------------------------   M - STEP   -------------------------- '''
            # Initializing the mean vector as a zero vector
            self.mean_vector = np.zeros((self.n_componets, len(X[0])))
            # Updating the mean vector
            for k in range(self.n_componets):
                for n in range(len(X)):
                    self.mean_vector[k] += self.r[n][k] * X[n]
            self.mean_vector = [1 / N[k] * self.mean_vector[k] for k in range(self.n_componets)]
            # Initiating the list of the covariance matrixes
            self.covariance_matrixes = [np.zeros((len(X[0]), len(X[0]))) for k in range(self.n_componets)]
            # Updating the covariance matrices
            for k in range(self.n_componets):
                self.covariance_matrixes[k] = np.cov(X.T, aweights=(self.r[:, k]), ddof=0)
            self.covariance_matrixes = [1 / N[k] * self.covariance_matrixes[k] for k in range(self.n_componets)]
            # Updating the pi list
            self.pi = [N[k] / len(X) for k in range(self.n_componets)]

    def predict(self, X):
        """
            The predicting function
                :param X: 2-d array numpy array
                    The data on which we must predict the clusters
        """
        probas = []
        for n in range(len(X)):
            probas.append([self.multivariate_normal(X[n], self.mean_vector[k], self.covariance_matrixes[k])
                           for k in range(self.n_componets)])
        cluster = []
        for proba in probas:
            cluster.append(self.comp_names[proba.index(max(proba))])
        return cluster

In [71]:

gm=GMM()
gm.fit(new_data,2)

In [44]:
gm.predict(new_data_test)